## Variation Autoencoder

`-` Encode / Decode

* Encode: 원래의 신호를 전달하기 위한 전자기 형태로 바꿈
> 처리를 하기 위해 신호/정보를 변환하는 것
* Decode: 전자기 데이터를 원래의 신호로 변환
> 변환된 데이터를 원래 처음의 입력으로 들어갔던 형태 그대로 바꾸는 것

`-` GPT

* Encode: 입력 프롬프트를 언어 모델이 이해할 수 있는 방식으로 처리
* Decode: 사람의 언어로 출력

`-` 베이지안 관점에서의 Encoder

* 이미지 $X$에서 이에 대한 정보에 해당하는 latent space에서 $Z$를 추출 $X \overset{\theta}{\to} Z$. $X$에서 $Z$로 정보를 축약.
* $Z$로부터 새로운 이미지 $X'$를 생성. 원래의 차원으로 돌림.
* 최근에 나온 방식들은 다른 방식으로 작동

> 정보를 축약하는 과정, encoding에서는 큰 문제가 없지만, decoding 과정에서 낮은 차원의 정보를 높은 차원의 결과물로 만들어내는 데에 어려움이 있음
>
> Difussion은 encoding 과정에서 정보의 차원을 줄이지 않고, 노이즈만 발생시킴. 이를 디노이징하는 과정을 decoding. 똑같은 크기의 변환을 수행하기 때문에, Bottleneck이 발생하지 않음. 대신 많은 Computing power가 필요함.

아무튼 인공지능에서 입/출력은 encode/decode로 일컫는다.

bottleneck hidden layer를 $z$, input/output을 $x$라 할 때.

* encoding process: $p_{\phi}(z~|~x)$
* decoding process: $p_{\theta}(x~|~z)$

$$P_{\phi}(z~|~x) \log P_{\theta}(x~|~z)$$

> $\log P_{\theta}(x~|~z)$는 가우시안 형태를 가정한다면 손실함수의 형태로 나타남. 이 경우 계산에 문제가 없음.
>
> $P(z~|~x)$는 그 형태를 정하기 어려움. 가우시안 $q_{\phi}(z)$로 근사하여 획득(ELBO) $\to$ $q$를 알고 있으므로 $z$로 인코딩한 다음 디코딩하면 됨.

### 1. Encoder and decoder

* context를 정리된 정보로 변환하고, 이를 다시 context로 변환
* Bottleneck hidden layer / Code / Latent Variable / Feature / Hidden representation

`-` input $x$와 output $x$는 다름.

* 단순 정보 전달에서도 encoding/decoding 과정에서 정보 손실이 생김
* 가장 간단한 예로 PCA.
* encoding은 차원 축소, decoding은 oversampling이라고 볼 수 있음

`-` DNN을 이용한 인코더

* $\log p_{\theta}(x~|~z) \propto (x - \mu(z))^2$
* $p(z~|~x) \approx q_{\phi}(z)$
> 계산식이 아닌 데이터만으로 $q_{\phi}(z)$를 추정할 수 있음

### Basic formulation

1. ELBO

$$\mathbb E_{q_{\phi}}[\log p_{\theta}(x^{(i)}~|~z)] - \text{KL}(q_{\phi}(z)~||~p_{\theta}(z))$$

where $q_{\phi}(z) ≡ q_{\phi}(z~|~x^{(i)})$

> $z$를 독립으로 근사
>
> 수식 기반으로 그래디언트를 계산하기에는 매우 어려움...

2. Monte Carlo gradient for the expectation

$$\mathbb E_{q_{\phi}}[f_{\theta}(z)] \approx \sum_{j=1}^M f_{\theta}(z_j)/M, ~ z_j \sim q_{\phi}$$

$\partial/ \partial \phi$에 대해서 근사된 수식을 미분하기가 매우 까다로움. $q_{\phi}(z)$가 사라지지 않도록 만들자.

$$\begin{align}
\frac{\partial q_{\phi}(z)}{\partial \phi} & = \frac{\partial \exp \log q_{\phi}(z)}{\partial \phi} \\
& = \exp \log q_{\phi}(z) \cdot \frac{\partial \log q_{\phi}(z)}{\partial \phi} \\
\therefore \nabla_{\phi} \mathbb E_{q_{\phi}} [f(z)] & = \mathbb E_{q_{\phi}}[f(z) \nabla_{\phi} \log q_{\phi} (z)] \approx \sum_{l=1}^L f(z^{(l)}) \nabla_{\phi} \log q_{\phi} (z^{(l)}) / L
\end{align}$$

where $z^{(l)} \sim q_{\phi}$

3. Reparameterization trick $z \sim q_{\phi}(z~|~x) \Leftarrow\Rightarrow g(\epsilon, x; \phi)$

$$X_i \sim N(\mu, \sigma^2) \Rightarrow X_i = \mu + \sigma \epsilon_i, ~ \epsilon_i \sim N(0, 1)$$

$$\begin{align}
\sum_{i=1}^M X_i / M & = \sum_{i=1}^M (\mu + \sigma \epsilon_i) / M \\
& = \sum_{i=1}^M \mu / M + \sigma \sum_{i=1}^M \epsilon_i / M \\
& = \mu + \sigma \bar \epsilon_M \\
\Rightarrow \frac{\partial \sum_{i=1}^M X_i / M}{\partial \sigma} & = \bar \epsilon_M
\end{align}$$

> Stochastic gradient $\bar epsilon_M$이 발생. 어차피 SGD에서 확률적인 매커니즘이 존재하는데, 이거 추가한다고 크게 달라지겠느냐?

* Stochastic Gradient Variational Bayes SGVB

$$\mathcal {\tilde L}^B = - \text{KL}(q_{\phi}(z~|~x^{(i)})~||~p_{\theta}(z)) + \frac{1}{L} \sum_{i=1}^L \log p_{\theta}(x^{(i)}~|~ z^{(i, l)})$$

where $z^{(i, l)} = g_{\phi} (\epsilon^{(l)}, x^{(i)})$ and $\epsilon^{(l)} \sim p(\epsilon)$. $z^{(i, l)} = \mu_{\phi} (x^{(i)}) + \sigma_{\phi} (x^{(i)}) \epsilon^{(l)}$

> $\phi$를 움직이면 $z^{(i, l)}$이 변경되고, 이게 손실함수 뒷부분을 변경 ($\theta$). 여기서부터 다시 $\phi$를 생성.

$p_{\theta}(x~|~z), \mu, \sigma$에 DNN을 걸어야 함.

`-` VAE

* VAE 단독으로 파인한 생성을 만드는 것은 불가능 -> difussion을 같이 넣으면 유용
* Denoising: 처음부터 잡음이 섞인 오브젝트를 삽입하면 선명해짐. 모형에 $\epsilon$이 있어서 그런가?
* 이상치 탐색: 정상 데이터는 잘 나오지만, 이상 데이터를 latent space에 삽입하면 비정상적인 결과가 나옴 $\to$ 탐색에 활용 가능

`-` 정리

* ELBO에서 $\theta$는 $p_{\theta}(x^{(i)}~|~z)$를 결정.
* posterior의 근사 $q_{\phi}(z) ≡ q_{\phi}(z~|~x^{(i)})$
> $q$를 알아냄으로써 $z$를 생성: encoder

* $z^{(i, l)}$을 reparametrization. $z = \epsilon \sigma + \mu$. $\phi = (\mu, \sigma)$
> $\phi$를 미분했을 때 확률변수가 나오므로, Stochastic Gradient

1. $z$는 $x$에 대해서 변화하고, 이는 $x$의 $\phi = (\mu, \sigma)$로 결정된다.
2. ㅁㄴㅇㄹ

`-` Advanced directions

1. Hybrid with GAN and diffusion. 단독으론 잘 안되고, 하이브리드 아키텍쳐를 구성하면 깔끔한 결과가 나온다.
2. Structured VAE: $z$에 correlation을 부여하는 등으로 원하는 방향으로 구축하는 방식. (fair representation or contextual regularization)
3. CoVAE (Consistency training): $z$ 도메인에서 통제. 특정 도메인의 값이 나오도록
4. LLM supported encoder and decoder

`-` Advantages and disadvantages

1. Probabilistic model, generation. 생성이 가능...은 함
2. General applications.
3. Blurr, loss of details, and computational costs.

### Advanced VAE

`-` conditional VAE

$$\mathbb E_{q_{\phi}} [\log p_{\theta} (x^{(i)})~|~z, y] - \text{KL}(q_{\theta}(z)~||~p_{\theta}(z))$$

where $q_{\phi}(z) ≡ q_{\phi}(z~|~x^{(i)}, y)$

> 레이블 정보를 추가적으로 제공
>
> $p_{\theta}(z)$를 prior라고 취급하면... $\text{KL}$ divergence를 줄이는 방향으로 최적화됨. 이미 정해둔 prior에 $q_{\phi}(z)$를 가깝도록 만듦.
>
> $\phi$를 찾아갈 때 출발점을 어떻게 설정하는지를 결정하는 느낌.

`-` $\beta$ VAE

* KL divergence penalty term에 붙이는 계수. 1보다 크게 만듦
* 학습을 안정화.

### KL divergence 계산

* KL divergence는 손으로 계산해서 수식을 집어넣어야 함.

$$\begin{align}
q_{\phi}(x) & = (2\pi\sigma^2)^{-\frac12} \exp \left( -\frac{1}{2\sigma^2}(x - \mu)^2 \right) \\
p_{\theta}(x) & = (2\pi)^{-\frac12} \exp \left( -\frac{1}{2}x^2 \right) \\ \\
& -\frac12 \log 2\pi\sigma^2 - \frac{1}{2\sigma^2}(x - \mu)^2 + \frac12 \log 2\pi + \frac12 x^2 \\
& \overset{\mathbb E_q}{\Rightarrow} -\frac12 \left\{ \log \sigma^2 + \mathbb E \left[\frac{1}{\sigma^2} (x - \mu)^2\right] - \mathbb E [x^2] \right\} \\
& = -\frac12 \left\{ \log \sigma^2 + \frac{1}{\sigma^2}\sigma^2 - (\mu^2 + \sigma^2) \right\} \\
& = \frac{1}{2} \left\{ \mu^2 + \sigma^2 - \log \sigma^2 - 1 \right\}
\end{align}$$